In [1]:
import imodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error, r2_score
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import *
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.pyplot as plt
import openml
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('.')
sys.path.append('./scripts')
from competing_methods_local import *
from simulations_util import *
from collections import Counter
from ucimlrepo import fetch_ucirepo

/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
selected_id = []
for task_id in [31, 10101, 3913, 3, 3917, 9957, 9946, 3918, 3903, 37, 9971, 9952, 3902,
                49, 43, 9978, 10093, 219, 9976, 14965, 6, 9977, 53, 11, 15, 16, 14,
                32, 3549, 12, 9981, 18, 28, 2074, 29, 45, 125922, 9960, 9964, 22, 2079,
                14969, 3560, 14952, 125920, 23, 3904, 3022, 9985, 9910, 14970, 3021, 
                3481, 7592, 3573, 146824, 146820, 146822, 146195, 146800, 146817, 146819,
                146821, 167119, 14954, 167141, 167140, 167120, 167125, 146825, 167124, 167121]:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    if X.shape[1] > 10 and y.nunique() == 2:
        selected_id.append(task_id)

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

In [11]:
# [31,
#  3913,
#  3,
#  3917,
#  9957,
#  9946,
#  3918,
#  3903,
#  3902,
#  43,
#  9978,
#  9976,
#  14965,
#  9977,
#  29,
#  14952,
#  125920,
#  3904,
#  9910,
#  3021,
#  7592,
#  146819,
#  14954,
#  167141,
#  167120,
#  167125]

In [4]:
def check_columns_with_few_distinct_values(df):
    """
    Check if any column in the DataFrame has fewer than 10 distinct values.

    Parameters:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    dict: A dictionary with column names as keys and the count of distinct values as values for columns with fewer than 10 distinct values.
    """
    result = {col: df[col].nunique() for col in df.columns if df[col].nunique() < 10}
    return result

for t_id in selected_id:
    task = openml.tasks.get_task(t_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    print(t_id)
    print(check_columns_with_few_distinct_values(X))

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

31
{'checking_status': 4, 'credit_history': 5, 'savings_status': 5, 'employment': 5, 'installment_commitment': 4, 'personal_status': 4, 'other_parties': 3, 'residence_since': 4, 'property_magnitude': 4, 'other_payment_plans': 3, 'housing': 3, 'existing_credits': 4, 'job': 4, 'num_dependents': 2, 'own_telephone': 2}
3913
{'lOCodeAndComment': 8}
3
{'bkblk': 2, 'bknwy': 2, 'bkon8': 2, 'bkona': 2, 'bkspr': 2, 'bkxbq': 2, 'bkxcr': 2, 'bkxwp': 2, 'blxwp': 2, 'bxqsq': 2, 'cntxt': 2, 'dsopp': 2, 'dwipd': 2, 'hdchk': 2, 'katri': 3, 'mulch': 2, 'qxmsq': 2, 'r2ar8': 2, 'reskd': 2, 'reskr': 2, 'rimmx': 2, 'rkxwp': 2, 'rxmsq': 2, 'simpl': 2, 'skach': 2, 'skewr': 2, 'skrxp': 2, 'spcop': 2, 'stlmt': 2, 'thrsk': 2, 'wkcti': 2, 'wkna8': 2, 'wknck': 2, 'wkovl': 2, 'wkpos': 2}
3917
{}
9957
{'V4': 4, 'V19': 3, 'V20': 4, 'V21': 4, 'V24': 2, 'V25': 2, 'V26': 4, 'V29': 2, 'V32': 8, 'V35': 8, 'V38': 8, 'V40': 5}
9946
{}
3918
{}
3903
{'PARAMETER_COUNT': 8}
3902
{'DECISION_DENSITY': 5, 'ESSENTIAL_DENSITY': 2, '

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

{'V7': 2, 'V8': 2, 'V9': 8, 'V15': 3, 'V16': 3, 'V17': 4, 'V18': 4, 'V23': 3, 'V24': 3, 'V31': 3, 'V32': 3, 'V39': 3, 'V40': 3, 'V41': 3, 'V42': 3, 'V47': 3, 'V48': 3, 'V49': 7, 'V50': 7, 'V55': 3, 'V56': 3, 'V63': 3, 'V64': 3, 'V71': 3, 'V72': 3, 'V73': 4, 'V74': 4, 'V79': 3, 'V80': 3, 'V81': 3, 'V82': 3, 'V83': 3, 'V84': 3, 'V85': 3, 'V86': 3, 'V87': 3, 'V88': 3, 'V92': 3, 'V96': 3, 'V100': 3, 'V104': 3, 'V108': 3, 'V112': 3, 'V116': 3}
29
{'A1': 2, 'A4': 3, 'A5': 3, 'A7': 9, 'A9': 2, 'A10': 2, 'A12': 2, 'A13': 3}
14952
{'having_IP_Address': 2, 'URL_Length': 3, 'Shortining_Service': 2, 'having_At_Symbol': 2, 'double_slash_redirecting': 2, 'Prefix_Suffix': 2, 'having_Sub_Domain': 3, 'SSLfinal_State': 3, 'Domain_registeration_length': 2, 'Favicon': 2, 'port': 2, 'HTTPS_token': 2, 'Request_URL': 2, 'URL_of_Anchor': 3, 'Links_in_tags': 3, 'SFH': 3, 'Submitting_to_email': 2, 'Abnormal_URL': 2, 'Redirect': 2, 'on_mouseover': 2, 'RightClick': 2, 'popUpWidnow': 2, 'Iframe': 2, 'age_of_domain

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

9910
{'D4': 5, 'D23': 2, 'D24': 5, 'D27': 2, 'D28': 2, 'D29': 7, 'D51': 2, 'D72': 2, 'D77': 7, 'D79': 6, 'D81': 4, 'D82': 5, 'D94': 2, 'D98': 5, 'D108': 7, 'D109': 6, 'D110': 7, 'D111': 7, 'D112': 5, 'D113': 6, 'D114': 6, 'D115': 6, 'D116': 5, 'D127': 7, 'D134': 7, 'D136': 8, 'D148': 8, 'D156': 6, 'D163': 9, 'D164': 7, 'D165': 7, 'D166': 6, 'D169': 3, 'D170': 2, 'D225': 3, 'D226': 6, 'D227': 8, 'D228': 5, 'D229': 5, 'D230': 3, 'D231': 3, 'D234': 6, 'D238': 5, 'D240': 5, 'D242': 3, 'D251': 3, 'D252': 3, 'D253': 3, 'D254': 4, 'D255': 5, 'D256': 4, 'D257': 8, 'D258': 4, 'D259': 4, 'D260': 9, 'D262': 8, 'D263': 5, 'D264': 5, 'D265': 8, 'D266': 4, 'D267': 4, 'D268': 8, 'D269': 4, 'D270': 5, 'D271': 3, 'D272': 5, 'D273': 4, 'D274': 3, 'D275': 3, 'D276': 5, 'D277': 5, 'D278': 3, 'D279': 3, 'D280': 5, 'D281': 5, 'D282': 6, 'D283': 5, 'D284': 3, 'D285': 3, 'D286': 3, 'D287': 3, 'D288': 6, 'D289': 6, 'D290': 4, 'D291': 5, 'D292': 3, 'D293': 4, 'D294': 5, 'D295': 4, 'D296': 9, 'D297': 4, 'D298': 

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

{}
14954
{'grain_screened': 2, 'ink_color': 1, 'proof_on_ctd_ink': 2, 'blade_mfg': 2, 'cylinder_division': 1, 'paper_type': 3, 'ink_type': 3, 'direct_steam': 2, 'solvent_type': 3, 'type_on_cylinder': 2, 'press_type': 4, 'press': 8, 'unit_number': 7, 'cylinder_size': 3, 'paper_mill_location': 5, 'plating_tank': 2, 'ESA_Amperage': 4, 'current_density': 7}
167141
{'area_code': 3, 'international_plan': 2, 'voice_mail_plan': 2}
167120
{}
167125
{'local': 2, 'url.images.buttons': 2, 'url.likesbooks.com': 2, 'url.www.slake.com': 2, 'url.hydrogeologist': 2, 'url.oso': 2, 'url.media': 2, 'url.peace.images': 2, 'url.blipverts': 2, 'url.tkaine.kats': 2, 'url.labyrinth': 2, 'url.advertising.blipverts': 2, 'url.images.oso': 2, 'url.area51.corridor': 2, 'url.ran.gifs': 2, 'url.express.scripts.com': 2, 'url.off': 2, 'url.cnet': 2, 'url.time.1998': 2, 'url.josefina3': 2, 'url.truluck.com': 2, 'url.clawnext.gif': 2, 'url.autopen.com': 2, 'url.tvgen.com': 2, 'url.pixs': 2, 'url.heartland.5309': 2, 'url.

In [73]:
task = openml.tasks.get_task(167132) #24 21
dataset = task.get_dataset()
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
X

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V391,V392,V393,V394,V395,V396,V397,V398,V399,V400
0,-0.35069,0.52721,-1.62470,-1.334000,-1.03020,1.247500,-0.98067,-1.558700,-0.061233,1.048200,...,0.437720,0.570960,-1.19840,2.168600,-0.30353,1.21620,-0.607070,0.503820,-0.45571,0.86283
1,-0.42537,-0.08818,0.84575,0.881570,1.35690,0.748940,-1.68620,0.461510,0.585700,0.775950,...,-0.016239,-0.582980,-0.35666,-0.638000,0.64106,-0.94488,-1.069700,-0.024029,0.28696,-0.12374
2,-1.22100,0.43960,-0.06303,0.709530,0.95079,-0.218740,-1.24850,-1.628600,-1.339700,-1.858500,...,1.985500,-0.042742,2.77110,1.549400,-1.75680,-0.91595,-0.078469,0.097137,-0.55111,0.69951
3,-0.30132,0.55221,0.13769,0.715290,0.99311,-0.169590,-0.93405,-0.948250,-0.750880,0.003107,...,1.172300,-0.103520,2.53710,2.096000,-1.43680,-1.05230,-0.070276,-0.539810,-1.10380,0.70562
4,0.31292,0.42762,0.50348,0.005489,-0.94274,-0.176190,-0.50775,-0.464450,0.694750,0.376040,...,-1.317400,1.060700,1.39660,0.096776,-1.18360,-0.37802,0.357150,1.278600,-0.40139,0.11640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,-0.73029,0.41857,-0.41746,-0.454210,0.71168,-0.052522,-1.86610,0.220370,0.488910,-2.072700,...,-0.229240,-0.586100,1.47180,0.263640,-0.68870,-1.10260,0.467970,0.481060,-0.29457,0.55751
3682,-0.29375,1.46840,-0.28178,-1.411000,-0.91399,1.427700,0.68553,-0.549750,0.025032,0.857950,...,1.569100,0.575650,0.58935,2.316300,1.00600,-0.15520,-0.343240,-0.023543,-1.57520,-0.37716
3683,0.94843,0.35538,-1.04010,0.292630,0.17207,-0.549300,-2.22700,0.111300,0.403860,0.000354,...,-2.006600,0.544540,2.01170,0.584910,-0.43648,-0.11118,0.197870,0.189580,1.04610,-0.30757
3684,1.49800,-0.66785,1.04800,-1.569300,-0.65241,1.957500,0.45265,0.807050,0.601570,0.813750,...,-0.648440,-0.453780,-0.29842,-0.641800,0.49518,-2.56460,-0.708690,0.369670,-0.49249,-0.40257


In [74]:
X.corr()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V391,V392,V393,V394,V395,V396,V397,V398,V399,V400
V1,1.000000,-0.025484,0.007321,-0.024754,0.019995,-0.033776,0.002130,0.026861,0.031530,-0.011264,...,-0.078765,-0.015606,-0.018076,-0.012064,-0.013935,0.027390,-0.053923,0.006831,0.012746,0.023348
V2,-0.025484,1.000000,-0.011090,-0.008517,0.011441,0.013951,-0.016316,-0.048533,0.054263,0.049980,...,0.018304,-0.000833,-0.074950,0.010569,0.031855,0.071843,0.035666,-0.014386,-0.021948,-0.025867
V3,0.007321,-0.011090,1.000000,0.040902,-0.026231,0.054115,-0.016726,0.054642,-0.009365,0.049920,...,-0.007808,-0.018980,-0.014035,-0.027265,-0.011710,0.016509,0.027035,0.040052,0.007197,-0.022531
V4,-0.024754,-0.008517,0.040902,1.000000,0.024803,0.052348,-0.020100,0.026886,-0.036326,-0.057389,...,0.048215,-0.019176,0.029862,0.003135,0.040274,-0.063876,0.032652,-0.003589,0.085359,0.039180
V5,0.019995,0.011441,-0.026231,0.024803,1.000000,0.059901,0.023468,0.023703,-0.009613,0.000563,...,0.026136,0.004351,0.004750,0.017301,0.014859,-0.027062,-0.005016,-0.013738,-0.026433,0.061233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V396,0.027390,0.071843,0.016509,-0.063876,-0.027062,-0.048760,-0.015260,-0.054308,-0.012781,0.012183,...,-0.088906,0.049948,-0.033490,-0.024500,-0.030513,1.000000,0.021794,0.017062,-0.042919,0.036200
V397,-0.053923,0.035666,0.027035,0.032652,-0.005016,0.025024,0.003494,-0.077680,-0.084940,0.013504,...,0.006726,-0.018770,0.049422,0.023129,0.059621,0.021794,1.000000,0.008375,-0.036323,-0.019340
V398,0.006831,-0.014386,0.040052,-0.003589,-0.013738,-0.053146,0.070723,0.008167,0.030048,-0.026903,...,0.019821,-0.017177,0.011847,0.029269,0.037992,0.017062,0.008375,1.000000,0.000471,0.020597
V399,0.012746,-0.021948,0.007197,0.085359,-0.026433,0.014360,-0.012202,0.081244,0.065330,0.041501,...,0.014511,-0.014519,-0.005678,0.010663,0.052851,-0.042919,-0.036323,0.000471,1.000000,-0.004082


In [75]:
# check if there are NA in X
X.isna().sum()

V1      0
V2      0
V3      0
V4      0
V5      0
       ..
V396    0
V397    0
V398    0
V399    0
V400    0
Length: 400, dtype: int64

In [76]:
y

0       Anomaly
1       Anomaly
2       Anomaly
3       Anomaly
4       Anomaly
         ...   
3681     Normal
3682     Normal
3683     Normal
3684     Normal
3685     Normal
Name: Target, Length: 3686, dtype: category
Categories (2, object): ['Normal' < 'Anomaly']

In [9]:
y

0       False
1        True
2        True
3        True
4        True
        ...  
2104    False
2105    False
2106    False
2107    False
2108    False
Name: defects, Length: 2109, dtype: bool

In [14]:
#Final selected ids
# task_ids = [3917, 9946, 43, 9978, 146819, 167120]

#3904 has missing

In [23]:
X = sample_real_data_X(source="openml", task_id = 9946, normalize=True) #361243
y = sample_real_data_y(source="openml", task_id = 9946)[0] #361243


# partial_linear_lss_model_random_feature(X, beta=1, heritability=0.4, sigma=None, s=5, error_seed=1, feature_seed=1, return_support=True)

/tmp/ipykernel_2365353/965025377.py:1: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  X = sample_real_data_X(source="openml", task_id = 9946, normalize=True) #361243
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/task.py:1

In [25]:
# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [19]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
prediction_model_ridge=RidgeCV(cv=5)
prediction_model_lasso=LassoCV(cv=5)
prediction_model_en=ElasticNetCV(cv=5)
prediction_model_ridge.fit(X_train, y_train)
prediction_model_lasso.fit(X_train, y_train)
prediction_model_en.fit(X_train, y_train)

ElasticNetCV(cv=5)

In [116]:
import numpy as np
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Generate synthetic data for demonstration
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Fit RidgeCV
ridge = RidgeCV(cv=5, scoring='neg_mean_squared_error')
ridge.fit(X_train, y_train)
ridge_cv_error = -1*ridge.best_score_

# Fit LassoCV
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_train, y_train)
lasso_cv_error = np.mean(lasso.mse_path_, axis=1).min()

# Fit ElasticNetCV
elasticnet = ElasticNetCV(cv=5,l1_ratio=[0.1,0.5,0.7,0.9,0.95,0.99,1], random_state=0) #1.0 mean Lasso
elasticnet.fit(X_train, y_train)
elasticnet_cv_error = np.mean(elasticnet.mse_path_, axis=-1).min()

# Select the best model
errors = {
    'Ridge': ridge_cv_error,
    'Lasso': lasso_cv_error,
    'ElasticNet': elasticnet_cv_error
}
best_model_name = min(errors, key=errors.get)
best_model = {'Ridge': ridge, 'Lasso': lasso, 'ElasticNet': elasticnet}[best_model_name]

# Output results
print(f"Cross-validation errors: {errors}")
print(f"Best model: {best_model_name}")


Cross-validation errors: {'Ridge': 0.010654942277324744, 'Lasso': 0.051026930270252416, 'ElasticNet': 0.051026930270252416}
Best model: Ridge


In [112]:
# compute ridge mse on training
y_pred_ridge = ridge.predict(X_train)
ridge_mse = mean_squared_error(y_train, y_pred_ridge)
print(f"Ridge MSE on training: {ridge_mse}")

Ridge MSE on training: 0.010030112843297695


In [103]:
ridge.alpha_

0.1

In [96]:
y_pred = ridge.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE of the best model ({best_model_name}): {test_mse}")


Test MSE of the best model (Ridge): 0.011753256598425667


In [97]:
y_pred = lasso.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE of the best model ({best_model_name}): {test_mse}")

Test MSE of the best model (Ridge): 0.04924854632962474


In [98]:
y_pred = elasticnet.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE of the best model ({best_model_name}): {test_mse}")


Test MSE of the best model (Ridge): 0.04924854632962474


In [59]:
elasticnet.mse_path_

array([[27659.92820947, 55476.76399669, 19610.87993246, 18128.58587661,
        34250.42098335],
       [27649.45447736, 55476.76399669, 19600.60305206, 18122.65835464,
        34238.77341336],
       [27638.25352671, 55476.76399669, 19589.61434195, 18116.325546  ,
        34226.31452032],
       [27626.2771315 , 55476.76399669, 19572.21877633, 18109.56151447,
        34212.99010942],
       [27608.78457058, 55476.11423712, 19551.97931941, 18102.33892804,
        34189.48562027],
       [27577.560285  , 55462.62941741, 19530.34377017, 18077.28936643,
        34163.0913197 ],
       [27542.24668077, 55448.21074146, 19507.22088418, 18045.81482274,
        34133.04929463],
       [27500.76777154, 55417.95457116, 19481.30314451, 18012.88527788,
        34091.4185811 ],
       [27456.44847862, 55365.94172476, 19445.30213641, 17977.70787523,
        34046.92442804],
       [27409.10612396, 55310.3219057 , 19403.87300231, 17940.13913054,
        33999.3808548 ],
       [27356.65765012, 55239.

In [56]:
ridge

RidgeCV(alphas=array([1.00000000e-03, 1.32571137e-03, 1.75751062e-03, 2.32995181e-03,
       3.08884360e-03, 4.09491506e-03, 5.42867544e-03, 7.19685673e-03,
       9.54095476e-03, 1.26485522e-02, 1.67683294e-02, 2.22299648e-02,
       2.94705170e-02, 3.90693994e-02, 5.17947468e-02, 6.86648845e-02,
       9.10298178e-02, 1.20679264e-01, 1.59985872e-01, 2.12095089e-01,
       2.81176870e-01, 3.72759372e-0...
       8.68511374e-01, 1.15139540e+00, 1.52641797e+00, 2.02358965e+00,
       2.68269580e+00, 3.55648031e+00, 4.71486636e+00, 6.25055193e+00,
       8.28642773e+00, 1.09854114e+01, 1.45634848e+01, 1.93069773e+01,
       2.55954792e+01, 3.39322177e+01, 4.49843267e+01, 5.96362332e+01,
       7.90604321e+01, 1.04811313e+02, 1.38949549e+02, 1.84206997e+02,
       2.44205309e+02, 3.23745754e+02, 4.29193426e+02, 5.68986603e+02,
       7.54312006e+02, 1.00000000e+03]),
        cv=5)

In [37]:
elasticnet.mse_path_.shape

(50, 5)

In [40]:
np.mean(-elasticnet.mse_path_, axis=1)

array([-3.10333666e+04, -3.10333666e+04, -3.10333666e+04, -3.10333666e+04,
       -3.10333666e+04, -3.10333666e+04, -3.10180564e+04, -3.09624488e+04,
       -3.08111232e+04, -3.05533421e+04, -3.01579596e+04, -2.96391496e+04,
       -2.89794054e+04, -2.81382460e+04, -2.70748168e+04, -2.57626316e+04,
       -2.41777229e+04, -2.23125076e+04, -2.01792954e+04, -1.78217808e+04,
       -1.53156291e+04, -1.27671919e+04, -1.02949589e+04, -8.01976058e+03,
       -6.02582118e+03, -4.37056910e+03, -3.06011553e+03, -2.07160753e+03,
       -1.36524652e+03, -8.73354547e+02, -5.45990427e+02, -3.34790877e+02,
       -2.02039879e+02, -1.20376935e+02, -7.09810175e+01, -4.15219904e+01,
       -2.41399926e+01, -1.39673575e+01, -8.05579856e+00, -4.63658290e+00,
       -2.66602450e+00, -1.53352287e+00, -8.83550198e-01, -5.11890751e-01,
       -2.98865021e-01, -1.76986637e-01, -1.07245665e-01, -6.74702087e-02,
       -4.46181271e-02, -3.14488829e-02])

In [21]:
prediction_model_ridge.mse_path_

AttributeError: 'RidgeCV' object has no attribute 'mse_path_'

In [20]:
prediction_model_ridge.best_score_

0.1345599900143149

In [104]:
from sklearn.linear_model import LassoCV
model = LassoCV(cv=5, max_iter=10000)
model.fit(X_train, y_train)

best_alpha_idx = np.where(model.alphas_ == model.alpha_)[0][0]

cv_errors = model.mse_path_
# Get the cross-validation errors for the best alpha
errors_for_best_alpha = cv_errors[best_alpha_idx, :]

# Compute the mean cross-validation error for the best alpha
mean_error_for_best_alpha = errors_for_best_alpha.mean()

print("Mean CV Error for the Best Alpha:", mean_error_for_best_alpha)
print(np.mean(model.mse_path_, axis=1).min())

Mean CV Error for the Best Alpha: 259.1724995334133


In [118]:
from sklearn.linear_model import ElasticNetCV
model = ElasticNetCV(cv=5,l1_ratio=[0.1, 0.5, 0.7, 0.9], max_iter=10000)
model.fit(X_train, y_train)
np.mean(model.mse_path_, axis=-1).min()

ElasticNetCV(cv=5, l1_ratio=[0.1, 0.5, 0.7, 0.9], max_iter=10000)

In [122]:
np.mean(model.mse_path_, axis=-1).min()

259.2471195818597

In [29]:
model =LogisticRegressionCV(penalty='l1', solver = 'saga', cv=5, max_iter=10000)
model.fit(X_train, y_train)
print(np.mean(model.scores_[1], axis=0).max())

0.9670329670329669


In [33]:
model.scores_[1]

array([[0.62637363, 0.62637363, 0.62637363, 0.94505495, 0.94505495,
        0.95604396, 0.93406593, 0.93406593, 0.93406593, 0.93406593],
       [0.62637363, 0.62637363, 0.62637363, 0.92307692, 0.92307692,
        0.94505495, 0.93406593, 0.92307692, 0.92307692, 0.92307692],
       [0.62637363, 0.62637363, 0.62637363, 0.97802198, 1.        ,
        0.98901099, 0.98901099, 0.98901099, 0.98901099, 0.98901099],
       [0.62637363, 0.62637363, 0.62637363, 0.93406593, 0.96703297,
        0.97802198, 0.96703297, 0.96703297, 0.96703297, 0.96703297],
       [0.62637363, 0.62637363, 0.62637363, 0.96703297, 0.98901099,
        0.96703297, 0.96703297, 0.96703297, 0.96703297, 0.96703297]])

In [30]:
# compute accuracy on the training set
y_train_pred = model.predict(X_train)
accuracy_train = np.mean(y_train == y_train_pred)
print('Training Accuracy:', accuracy_train)

Training Accuracy: 0.989010989010989


In [27]:
model =LogisticRegressionCV(penalty='l2', cv=5, max_iter=10000)
model.fit(X_train, y_train)
print(np.mean(model.scores_[1], axis=0).max())

0.9736263736263737


In [28]:
model =LogisticRegressionCV(penalty='elasticnet',l1_ratios=[0.1, 0.5, 0.7, 0.9], solver = 'saga', cv=5, max_iter=10000)
model.fit(X_train, y_train)
print(np.mean(model.scores_[1], axis=0).max())

0.9758241758241759


In [79]:
np.mean(model.scores_[1], axis=0).max()

0.8625

In [80]:
np.mean(model.scores_[1], axis=0)

array([[0.843125, 0.843125, 0.843125, 0.843125],
       [0.843125, 0.843125, 0.843125, 0.843125],
       [0.853125, 0.84625 , 0.843125, 0.843125],
       [0.8625  , 0.856875, 0.855625, 0.855625],
       [0.85875 , 0.86    , 0.859375, 0.859375],
       [0.854375, 0.854375, 0.85375 , 0.853125],
       [0.851875, 0.851875, 0.851875, 0.851875],
       [0.851875, 0.851875, 0.851875, 0.851875],
       [0.851875, 0.851875, 0.851875, 0.851875],
       [0.851875, 0.851875, 0.851875, 0.851875]])

In [81]:
model.Cs_

array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])

In [82]:
model.C_

array([0.04641589])

In [58]:
optimal_c_index = np.where(model.Cs_ == model.C_[0])[0][0]  # Optimal C value index
optimal_l1_index = np.argmax([score.mean() for score in model.scores_[1]])  # Optimal l1_ratio index

# Compute the mean cross-validated score for the optimal C and l1_ratio
mean_cv_score_optimal = model.scores_[1][optimal_l1_index][:, optimal_c_index].mean()

# Print results
print(f"Mean cross-validated score (optimal): {mean_cv_score_optimal}")
print(f"Test set score: {model.score(X_test, y_test)}")

IndexError: index 3 is out of bounds for axis 1 with size 3

In [57]:
optimal_l1_index

NameError: name 'optimal_l1_index' is not defined

In [53]:
model.Cs_

array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])

In [54]:
model.C_

array([0.04641589])

In [51]:
model.scores_[1]

array([[[0.840625, 0.840625, 0.840625],
        [0.840625, 0.840625, 0.840625],
        [0.85625 , 0.846875, 0.840625],
        [0.859375, 0.859375, 0.859375],
        [0.853125, 0.853125, 0.853125],
        [0.85    , 0.85    , 0.846875],
        [0.84375 , 0.84375 , 0.84375 ],
        [0.84375 , 0.84375 , 0.84375 ],
        [0.84375 , 0.84375 , 0.84375 ],
        [0.84375 , 0.84375 , 0.84375 ]],

       [[0.84375 , 0.84375 , 0.84375 ],
        [0.84375 , 0.84375 , 0.84375 ],
        [0.84375 , 0.84375 , 0.84375 ],
        [0.8625  , 0.846875, 0.84375 ],
        [0.853125, 0.85625 , 0.859375],
        [0.85    , 0.85    , 0.85    ],
        [0.85    , 0.85    , 0.85    ],
        [0.85    , 0.85    , 0.85    ],
        [0.85    , 0.85    , 0.85    ],
        [0.85    , 0.85    , 0.85    ]],

       [[0.84375 , 0.84375 , 0.84375 ],
        [0.84375 , 0.84375 , 0.84375 ],
        [0.840625, 0.846875, 0.84375 ],
        [0.85625 , 0.84375 , 0.846875],
        [0.85625 , 0.85625 , 0.85625

In [ ]:
model =LogisticRegressionCV(penalty='l1', solver = 'saga', cv=5, max_iter=10000)
model.fit(X, y)
optimal_c_index = np.where(model.Cs_ == model.C_[0])[0][0]
mean_cv_score_optimal = model.scores_[1][:, optimal_c_index].mean()
print(mean_cv_score_optimal)

In [20]:
optimal_c_index = np.where(model.Cs_ == model.C_[0])[0][0]
mean_cv_score_optimal = model.scores_[1][:, optimal_c_index].mean()

Mean CV score for the optimal C: 0.8585


In [ ]:
model.Cs_

In [18]:
# 'scores_' is a dictionary where keys are the classes and values are arrays of cross-validation scores
cv_scores = model.scores_

# For binary classification, get scores for the positive class (assumes classes are 0 and 1)
positive_class_scores = cv_scores[1]  # Replace '1' with the appropriate class if needed

# Compute mean cross-validation score for each fold
mean_cv_scores = positive_class_scores.mean(axis=0)
print("Mean CV scores for each fold:", mean_cv_scores)

Mean CV scores for each fold: [0.85725 0.8535  0.84325 0.85375 0.85475]


In [16]:
model.coefs_paths_

{1.0: array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        , -0.59743822],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        , -0.87520469],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        , -1.71796625],
         ...,
         [-0.53351644, -2.25940196, -0.1795186 , ...,  0.79067552,
           0.82586326, -2.19200146],
         [-0.5372369 , -2.27201086, -0.17950076, ...,  0.79415234,
           0.83763395, -2.19219036],
         [-0.53777537, -2.273832  , -0.17949235, ...,  0.79465248,
           0.83933726, -2.19221569]],
 
        [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        , -0.56040027],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        , -0.82944166],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        , -1.7196825 ],
         ...,
         [ 0.8874702 , -0.41

In [13]:
ridge.cv_values_

array([[1.94314928, 1.90211883, 1.54589127],
       [1.17325286, 1.18581533, 1.2852184 ],
       [1.89360901, 1.84943164, 1.53011218],
       ...,
       [2.01460368, 2.01930823, 2.03673588],
       [6.60199672, 6.59819412, 6.62457306],
       [0.06044319, 0.07083127, 0.20115005]])

In [14]:
ridge.alpha_

0.1

In [ ]:
assert False

In [ ]:
y[15]

In [ ]:
0.242974+0.33677068-1.2268368+2

In [ ]:
support

In [ ]:
X[0]

In [ ]:
0.5324-0.404-0.5635

In [ ]:
y[0]

In [ ]:
from sklearn.linear_model import LassoCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf.fit(X_train, y_train)
rf_plus_base_lasso = RandomForestPlusRegressor(rf_model=rf, prediction_model=LassoCV(cv=5, max_iter=8000))
rf_plus_base_lasso.fit(X_train, y_train)

In [ ]:
rf_plus_alo_mdi = RFPlusMDI(rf_plus_base_lasso, mode = 'only_k', evaluate_on="all")
local_fi_score_train = rf_plus_alo_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)
local_fi_score_bootstrap_train = rf_plus_alo_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True, bootstrap=True)

In [ ]:
local_fi_score_train[4]

In [ ]:
roc_auc_score(support, local_fi_score_train[4])

In [ ]:
local_fi_score_bootstrap_train.shape

In [ ]:
local_fi_score_bootstrap_train[150, :,:].shape

In [ ]:
roc_auc_score(support, np.mean(np.argsort(np.argsort(np.abs(local_fi_score_bootstrap_train[10, :,:]), axis=0), axis=0), axis=1))

In [ ]:
np.argsort(np.argsort(np.abs(local_fi_score_bootstrap_train[150, :,:])))

In [ ]:
X.shape

In [ ]:
np.mean(local_fi_score_bootstrap_train[2, :,:], axis=-1)

In [ ]:
treeshap_train, _ = tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf, mode="absolute")

In [ ]:
auroc = []
for i in range(local_fi_score_train.shape[0]):
    fi_data_i = local_fi_score_train[i]    
    auroc.append(roc_auc_score(support, fi_data_i))
print(np.mean(auroc))

In [ ]:
auroc_shap = []
for i in range(treeshap_train.shape[0]):
    fi_data_i = treeshap_train[i]   
    auroc_shap.append(roc_auc_score(support, fi_data_i))
print(np.mean(auroc_shap))

In [ ]:
auroc_bootstrap = []
for i in range(local_fi_score_bootstrap_train.shape[0]):
    fi_data_i = local_fi_score_bootstrap_train[i]    
    auroc_bootstrap.append(roc_auc_score(support, fi_data_i))
print(np.mean(auroc_bootstrap))

In [ ]:
auroc_bootstrap[4]

In [ ]:
auroc[4]

In [ ]:
auroc_bootstrap = np.array(auroc_bootstrap)
auroc = np.array(auroc)

indices = auroc_bootstrap > auroc
gains = auroc_bootstrap[indices] - auroc[indices]
print("Gains:", sum(gains))
print("Number of gains:", sum(indices))

In [ ]:
indices = auroc_bootstrap < auroc
gains = auroc_bootstrap[indices] - auroc[indices]
print("Gains:", sum(gains))
print("Number of gains:", sum(indices))

In [ ]:
gains[4]

In [ ]:
auroc_bootstrap[4]

In [ ]:
auroc[4]

In [ ]:
np.argsort(-1*support)

In [ ]:
np.argsort(-1*local_fi_score_train[4])

In [ ]:
np.argsort(-1*local_fi_score_bootstrap_train[4])

In [ ]:
indices = auroc_bootstrap ==  auroc
gains = auroc_bootstrap[indices] - auroc[indices]
print("Gains:", sum(gains))
print("Number of gains:", sum(indices))

In [ ]:
sum(auroc)

In [ ]:
sum(auroc_bootstrap)

In [ ]:
print(sum(np.array(auroc_bootstrap) < np.array(auroc)))

In [ ]:
for i in range(local_fi_score_train.shape[0]):
    if auroc_bootstrap[i] > auroc[i]:
        print(i)
        print(auroc_bootstrap[i] - auroc[i])

In [ ]:
assert False

In [ ]:
np.abs(rf_plus.estimators_[0].coef_)[:20]

In [ ]:
rf_plus.estimators_[0].loo_coefficients_[:, :-1][0][:20]

In [ ]:
np.argsort(-1*np.argsort(np.argsort(np.abs(rf_plus.estimators_[0].coef_))))

In [ ]:
np.argsort(-1*np.argsort(np.argsort(np.abs(rf_plus.estimators_[0].loo_coefficients_[:, :-1][2]))))

In [ ]:
sum = 0
for i in range(X_train.shape[0]):
    sort_coef = np.argsort(-1 * np.argsort(np.argsort(np.abs(rf_plus.estimators_[0].coef_))))
    sort_loo_coefficients = np.argsort(-1 * np.argsort(np.argsort(np.abs(rf_plus.estimators_[0].loo_coefficients_[:, :-1][i]))))
    if np.array_equal(sort_coef[:5], sort_loo_coefficients[:5]):
        sum += 1
print(sum)


In [ ]:
assert False

In [ ]:
rf_plus.estimators_[0].loo_coefficients_[:, :-1][0]

In [ ]:
np.argsort(np.abs(rf_plus.estimators_[0].coef_[:, :-1][0]))

In [ ]:
np.argsort(np.abs(rf_plus.estimators_[0].loo_coefficients_[:, :-1][0]))

In [ ]:
rf_plus_alo_mdi = AloRFPlusMDI(rf_plus, mode = 'only_k', evaluate_on="all")
local_fi_score_train = rf_plus_alo_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

In [ ]:
rf_plus_mdi = RFPlusMDI(rf_plus, mode = 'only_k', evaluate_on="all")
local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

In [ ]:
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True, bootstrap=True)

In [ ]:
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True, bootstrap=False)

In [ ]:
local_fi_score_train

In [ ]:
#plot y
plt.hist(y)
plt.show()

In [ ]:
y = sample_real_data_y(source="openml", task_id = 361247)

In [ ]:
y

In [ ]:
"openml_361242" #"openml_361243" #"openml_361247" #"openml_361251" "openml_361253" "openml_361254" "openml_361256" "openml_361259" "openml_361260""openml_361622"
for task_id in [361242, 361243, 361247, 361251, 361253, 361254, 361256, 361259, 361260, 361622]:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    print(task_id)
    if X.shape[0] > 2000:
        print(int(2000 * 0.66))
    else:
        print(int(X.shape[0]*0.66))

In [ ]:
# for task_id in [361236, 361237, 361235, 361234, 361247, 361243, 361242, 361241, 361244, 361252, 361251, 361249, 361250, 361255, 361253, 361260, 
#                 361259, 361256, 361257, 361261, 361254, 361258, 361264, 361617, 361618, 361269, 361267, 361266, 361268, 361272,
#                 361616, 361619, 361621, 361623, 361622]:
#     task = openml.tasks.get_task(task_id)
#     dataset = task.get_dataset()
#     X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
#     X = pd.DataFrame(X, columns=attribute_names[:-1])
#     if X.shape[1] > 10:
#         print(task_id)
#         print(X.shape[0]*0.66)

In [ ]:
# id with more than 10 features
# selected_id = [361247, 361243, 361242, 361252, 361251, 361253, 361260, 361259, 361256, 361254, 361618, 161266, 361268,
#                361272, 361616, 361619, 361622]
# print(len(selected_id))

In [ ]:
# def check_columns_with_few_distinct_values(df):
#     """
#     Check if any column in the DataFrame has fewer than 10 distinct values.

#     Parameters:
#     df (pd.DataFrame): The input DataFrame.

#     Returns:
#     dict: A dictionary with column names as keys and the count of distinct values as values for columns with fewer than 10 distinct values.
#     """
#     result = {col: df[col].nunique() for col in df.columns if df[col].nunique() < 10}
#     return result

# for t_id in selected_id:
#     task = openml.tasks.get_task(t_id)
#     dataset = task.get_dataset()
#     X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
#     X = pd.DataFrame(X, columns=attribute_names[:-1])
#     print(t_id)
#     print(check_columns_with_few_distinct_values(X))

In [ ]:
# id with no categorical features
selected_id = [361247, 361243, 361242, 361251, 361253, 361260, 361259, 361256, 361254, 361622]
print(len(selected_id))

In [ ]:
# check if there are any missing values
for t_id in selected_id:
    task = openml.tasks.get_task(t_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    print(t_id)
    print(X.isnull().sum().sum())

# task = openml.tasks.get_task(t_id)
# dataset = task.get_dataset()
# X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")
# # X = pd.DataFrame(X, columns=attribute_names[:-1])
# X

In [ ]:
task = openml.tasks.get_task(361622)
dataset = task.get_dataset()
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")
# X = pd.DataFrame(X, columns=attribute_names[:-1])
y

In [ ]:
np.random.seed(4307)
np.random.choice(X.shape[0], 20, replace=False)

In [ ]:
np.random.seed(4307)
np.random.choice(X.shape[0], 20, replace=False)

In [ ]:
np.random.seed(4307)
np.random.choice(X.shape[0], 20, replace=False)

In [ ]:
y.shape[0]

In [ ]:
np.log(y)

In [ ]:
# Plot y distribution
plt.hist(y)
plt.show()

In [ ]:
y = np.log(y)
plt.hist(y)
plt.show()

In [ ]:
y

In [ ]:
# Turn y into a dataframe
y = pd.DataFrame(y, columns=['target'])

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_no_standardize = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf_no_standardize.fit(X_train, y_train)
rf_plus_base_no_standardize = RandomForestPlusRegressor(rf_model=rf_no_standardize)
rf_plus_base_no_standardize.fit(X_train, y_train)

In [ ]:
assert False

In [ ]:
def preprocessing_data_X(X):
    categorical_cols = X.select_dtypes(include=["object", "category"]).columns
    numerical_cols = X.select_dtypes(include=["number"]).columns
    if X[numerical_cols].isnull().any().any():
        num_imputer = SimpleImputer(strategy="mean")
        X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
    if len(categorical_cols) > 0 and X[categorical_cols].isnull().any().any():
        # Convert categorical columns to string to ensure consistent types
        X[categorical_cols] = X[categorical_cols].astype(str)
        cat_imputer = SimpleImputer(strategy="most_frequent")
        X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
    if len(categorical_cols) > 0:
        encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        X_categorical = encoder.fit_transform(X[categorical_cols])
        X_categorical_df = pd.DataFrame(
            X_categorical,
            columns=encoder.get_feature_names_out(categorical_cols),
            index=X.index
        )
        X = pd.concat([X[numerical_cols], X_categorical_df], axis=1)
    else:
        X = X[numerical_cols]
    X = X.to_numpy()
    if X.shape[0]>2000:
        X = X[:2000,:]
    return X

In [ ]:
# Temperature 925
dataset = fetch_ucirepo(id=857)
X = dataset.data.features
X

In [ ]:
X.select_dtypes(include=["object", "category"]).nunique()

In [ ]:
# Parkinsons 189
dataset = fetch_ucirepo(id=189)
X = dataset.data.features
X

In [ ]:
X.select_dtypes(include=["object", "category"]).nunique()

In [ ]:
X = sample_real_data_X(source="openml", data_id=1245)
y = sample_real_data_y(source="openml", data_id=1245, return_support=False)

In [ ]:
X = sample_real_data_X(source="openml", data_id=1245)
y = sample_real_data_y(source="openml", data_id=1245, return_support=False)

In [ ]:
dataset = openml.datasets.get_dataset(1245)
X, _, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
X = pd.DataFrame(X, columns=attribute_names[:-1])

In [ ]:
X

In [ ]:
X["histology"].value_counts()

In [ ]:
X.select_dtypes(include=["object", "category"]).columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

categorical_cols = X.select_dtypes(include=["object", "category"]).columns
numerical_cols = X.select_dtypes(include=["number"]).columns
if X[numerical_cols].isnull().any().any():
    num_imputer = SimpleImputer(strategy="mean")
    X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
if len(categorical_cols) > 0 and X[categorical_cols].isnull().any().any():
    X[categorical_cols] = X[categorical_cols].astype(str)
    cat_imputer = SimpleImputer(strategy="most_frequent")
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
if len(categorical_cols) > 0:
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    X_categorical = encoder.fit_transform(X[categorical_cols])
    X_categorical_df = pd.DataFrame(
        X_categorical,
        columns=encoder.get_feature_names_out(categorical_cols),
        index=X.index
    )

In [ ]:
X_categorical_df

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

def preprocessing_data_X(X):
    categorical_cols = X.select_dtypes(include=["object", "category"]).columns
    numerical_cols = X.select_dtypes(include=["number"]).columns
    if X[numerical_cols].isnull().any().any():
        num_imputer = SimpleImputer(strategy="mean")
        X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
    if len(categorical_cols) > 0 and X[categorical_cols].isnull().any().any():
        X[categorical_cols] = X[categorical_cols].astype(str)
        cat_imputer = SimpleImputer(strategy="most_frequent")
        X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
    if len(categorical_cols) > 0:
        encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        X_categorical = encoder.fit_transform(X[categorical_cols])
        X_categorical_df = pd.DataFrame(
            X_categorical,
            columns=encoder.get_feature_names_out(categorical_cols),
            index=X.index
        )
        X = pd.concat([X[numerical_cols], X_categorical_df], axis=1)
    else:
        X = X[numerical_cols]
    X = X.to_numpy()
    if X.shape[0]>2000:
        X = X[:2000,:]
    return X


In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_no_standardize = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf_no_standardize.fit(X_train, y_train)
rf_plus_base_no_standardize = RandomForestPlusRegressor(rf_model=rf_no_standardize)
rf_plus_base_no_standardize.fit(X_train, y_train)

In [ ]:
# print accuracy
print("RF+ accuracy: ", r2_score(y_test, rf_plus_base_no_standardize.predict(X_test)))
print("RF accuracy: ", r2_score(y_test, rf_no_standardize.predict(X_test)))

In [ ]:
shap_score_no_standardize = np.argsort(-1*tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf_no_standardize, mode="absolute")[0])
shap_score_no_standardize 

In [ ]:
lmdi_no_standardize = np.argsort(-1*LFI_evaluation_AloRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_base_no_standardize, mode="absolute")[0])
lmdi_no_standardize

In [ ]:
# standardize data
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf.fit(X_train, y_train)
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X_train, y_train)
# print accuracy
print("RF+ accuracy: ", r2_score(y_test, rf_plus_base.predict(X_test)))
print("RF accuracy: ", r2_score(y_test, rf.predict(X_test)))

In [ ]:
shap_score = np.argsort(-1*tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf, mode="absolute")[0])
shap_score

In [ ]:
lmdi = np.argsort(-1*LFI_evaluation_AloRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_base, mode="absolute")[0])
lmdi

In [ ]:
# assert 

In [ ]:
np.array_equal(lmdi, lmdi_no_standardize)

In [ ]:
difference_mask = lmdi != lmdi_no_standardize
print("Indices of differing elements:", np.argwhere(difference_mask))

In [ ]:
np.array_equal(shap_score, shap_score_no_standardize)

In [ ]:
difference_mask = shap_score != shap_score_no_standardize
print("Indices of differing elements:", np.argwhere(difference_mask))

In [ ]:
shap_score[5]

In [ ]:
shap_score_no_standardize[5]

In [ ]:
tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf, mode="absolute")[0]

In [ ]:
tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf_no_standardize, mode="absolute")[0]

In [ ]:
# Create heatmap
plt.figure(figsize=(8, 6))
plt.imshow(shap_score_no_standardize, cmap='viridis', aspect='auto')  # Choose colormap as needed
plt.colorbar(label='Values')  # Add a colorbar
plt.title('Matrix Heatmap')  # Add title
plt.xlabel('Columns')  # Add x-axis label
plt.ylabel('Rows')  # Add y-axis label
plt.show()

In [ ]:
# row_tuples = [tuple(row) for row in shap_score]
# row_counts = Counter(row_tuples)
# for row, count in row_counts.items():
#     print(f"Row: {row}, Count: {count}")

In [ ]:
# Create heatmap
plt.figure(figsize=(8, 6))
plt.imshow(lmdi, cmap='viridis', aspect='auto')  # Choose colormap as needed
plt.colorbar(label='Values')  # Add a colorbar
plt.title('Matrix Heatmap')  # Add title
plt.xlabel('Columns')  # Add x-axis label
plt.ylabel('Rows')  # Add y-axis label
plt.show()

In [ ]:
row_tuples = [tuple(row) for row in lmdi]
row_counts = Counter(row_tuples)
for row, count in row_counts.items():
    print(f"Row: {row}, Count: {count}")

In [ ]:
from ucimlrepo import fetch_ucirepo
dataset = fetch_ucirepo(id=10)
dataset.data.targets

In [ ]:
# apply train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf.fit(X_train, y_train)
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X_train, y_train)

In [ ]:
train_data, test_data = LFI_evaluation_RFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_base, mode="absolute")

In [ ]:
train_data[0]

In [ ]:
np.argsort(-1*train_data[0])[:3].tolist()

In [ ]:
assert False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example NumPy array
data = LFI_evaluation_RFPlus_all_ranking_retrain(X_train, y_train, fit=rf_plus_base, mode="absolute")
data = np.argsort(data, axis=1)  # Sort the indices of the features

# Adjust the figure size
plt.figure(figsize=(12, 6))  # Width = 12 inches, Height = 6 inches
plt.imshow(data, cmap='viridis', interpolation='nearest', aspect='auto')
plt.colorbar()  # Add a color bar to show the scale
plt.title("Heatmap of NumPy Array")
plt.show()



In [ ]:
LFI_evaluation_RFPlus_all_l2_norm_ranking_retrain(X_train, y_train, fit=rf_plus_base, mode="absolute")

In [ ]:
rf_plus_mdi = RFPlusMDI(rf_plus_base, evaluate_on="all")
temp1 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)
temp_10 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, leaf_average=True)
temp2 = rf_plus_mdi.explain_linear_partial_error_metric(X=X_train, y=y_train)
temp3 = rf_plus_mdi.explain_linear_partial_error_metric(X=X_train, y=y_train, leaf_average=True)
temp4 = rf_plus_mdi.explain_linear_partial_error_metric(X=X_train, y=y_train, ranking=True)


In [ ]:
from collections import defaultdict
leaf_indices = rf.apply(X_train).flatten()
leaf_mapping = defaultdict(list)
for sample_idx, leaf_idx in enumerate(leaf_indices):
    leaf_mapping[leaf_idx].append(sample_idx)
leaf_mapping[20]

In [ ]:
temp3[148]

In [ ]:
temp3[82]

In [ ]:
temp1[0]

In [ ]:
(y_train[0] - rf_plus_base.predict(X_train[0].reshape(1, -1)) + temp1[0])**2

In [ ]:
temp2[0]

In [ ]:
temp.shape

In [ ]:
y_train.shape

In [ ]:
result = (temp - y_train[:, np.newaxis, np.newaxis])**2

In [ ]:
y_train[:, np.newaxis, np.newaxis].shape

In [ ]:
y_train[20]

In [ ]:
(0.1676066)**2

In [ ]:
temp[20]

In [ ]:
result[20]

In [ ]:
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X_train, y_train)
rf_plus_base.score(X_test, y_test)

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
parkinsons_telemonitoring = fetch_ucirepo(id=189) 
  
# data (as pandas dataframes) 
X = parkinsons_telemonitoring.data.features 
y = parkinsons_telemonitoring.data.targets 


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

categorical_cols = X.select_dtypes(include=["object", "category"]).columns
numerical_cols = X.select_dtypes(include=["number"]).columns

# Step 2: Handle missing values (if any)
# Check if there are missing values in the numerical columns
if X[numerical_cols].isnull().any().any():
    # Impute missing values in numerical columns with the mean
    num_imputer = SimpleImputer(strategy="mean")
    X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])

# Check if there are missing values in the categorical columns
if len(categorical_cols) > 0 and X[categorical_cols].isnull().any().any():
    # Convert categorical columns to string to ensure consistent types
    X[categorical_cols] = X[categorical_cols].astype(str)

    # Impute missing values in categorical columns with the most frequent value
    cat_imputer = SimpleImputer(strategy="most_frequent")
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

# Step 3: Encode categorical variables using OneHotEncoder (if any categorical columns)
if len(categorical_cols) > 0:
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    X_categorical = encoder.fit_transform(X[categorical_cols])

    # Convert encoded categorical data back to DataFrame
    X_categorical_df = pd.DataFrame(
        X_categorical,
        columns=encoder.get_feature_names_out(categorical_cols),
        index=X.index
    )

    # Step 4: Concatenate numerical columns and the encoded categorical DataFrame
    X = pd.concat([X[numerical_cols], X_categorical_df], axis=1)
else:
    # If no categorical columns, we just use the numerical columns
    X = X[numerical_cols]
X = X.to_numpy()

In [ ]:
X

In [ ]:
if y.to_numpy().shape[1] > 1:
    y = y.iloc[:, 0].to_numpy().flatten()
else:
    y = y.to_numpy().flatten()

In [ ]:
# Fit a random forest model
rf = RandomForestRegressor(n_estimators=100, max_depth=5)
rf.fit(X, y)
rf.score(X, y)

In [ ]:
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X, y)
rf_plus_base.score(X, y)

In [ ]:
# X, y, _ = imodels.get_clean_dataset("diabetes")
X, y, _ = imodels.get_clean_dataset("diabetes_regr")

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
## Debug
# RF Regressor
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
est.fit(X_train, y_train)

# RFplus default(fit on all)
rf_plus_base = RandomForestPlusRegressor(rf_model=est)
rf_plus_base.fit(X_train, y_train)

rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
temp = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

In [ ]:
temp.shape

In [ ]:
temp[0,0,:].shape

In [ ]:
r2_score([y_train[0]]*100, temp[1,1,:])

In [ ]:
# # RF Regressor
# est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
# est.fit(X_train, y_train)

# # RFplus default(fit on all)
# rf_plus_base = RandomForestPlusRegressor(rf_model=est)
# rf_plus_base.fit(X_train, y_train)

# # RFplus oob 
# rf_plus_base_oob = RandomForestPlusRegressor(rf_model=est, fit_on="oob")
# rf_plus_base_oob.fit(X_train, y_train)

# #RFplus inbag RF
# rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=est, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
# rf_plus_base_inbag.fit(X_train, y_train)

In [ ]:
# RF Classifier
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est.fit(X_train, y_train)

# RFplus default(fit on all)
rf_plus_base = RandomForestPlusClassifier(rf_model=est)
rf_plus_base.fit(X_train, y_train)

# RFplus oob 
rf_plus_base_oob = RandomForestPlusClassifier(rf_model=est, fit_on="oob")
rf_plus_base_oob.fit(X_train, y_train)

rf_plus_base_inbag = RandomForestPlusClassifier(rf_model=est, include_raw=False, fit_on="inbag")
rf_plus_base_inbag.fit(X_train, y_train)

# #RFplus inbag RF
# est_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
# est_regressor.fit(X_train, y_train)
# rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=est_regressor, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
# rf_plus_base_inbag.fit(X_train, y_train)

In [ ]:
# Inbag LMDI+
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

# OOB LMDI+
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

# ALL LMDI+
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

# Inbag LMDI+ l2 norm with sign
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# OOB LMDI+ l2 norm with sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# ALL LMDI+ l2 norm with sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# Inbag LMDI+ l2 norm without sign
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# OOB LMDI+ l2 norm without sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# ALL LMDI+ l2 norm without sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# Inbag LMDI+ with ranking then average
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

# OOB LMDI+ with ranking then average
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

# ALL LMDI+ with ranking then average
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

In [ ]:
# Inbag LMDI+ l2 norm with sign
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# OOB LMDI+ l2 norm with sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# ALL LMDI+ l2 norm with sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

In [ ]:
# Inbag LMDI+ l2 norm without sign
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# OOB LMDI+ l2 norm without sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# ALL LMDI+ l2 norm without sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

In [ ]:
# Inbag LMDI+ with ranking then average
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

# OOB LMDI+ with ranking then average
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

# ALL LMDI+ with ranking then average
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

In [ ]:
# RF Classifier
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est.fit(X_train, y_train)

# RFplus default(fit on all)
rf_plus_base = RandomForestPlusClassifier(rf_model=est)
rf_plus_base.fit(X_train, y_train)

# RFplus oob 
rf_plus_base_oob = RandomForestPlusClassifier(rf_model=est, fit_on="oob")
rf_plus_base_oob.fit(X_train, y_train)

#RFplus inbag RF
est_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est_regressor.fit(X_train, y_train)
rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=est_regressor, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
rf_plus_base_inbag.fit(X_train, y_train)

In [ ]:
X_test_pred = est.predict(X_test)
print("R2 score of RF: ", r2_score(y_test, X_test_pred))

In [ ]:
X_test_pred = rf_plus_base.predict(X_test)
print("R2 score of RF+: ", r2_score(y_test, X_test_pred))

In [ ]:
if X_train.shape[0] > 100:
    indices_train = np.random.choice(X_train.shape[0], 100, replace=False)
    X_train_subset = X_train[indices_train]
    y_train_subset = y_train[indices_train]
else:
    indices_train = np.arange(X_train.shape[0])
    X_train_subset = X_train
    y_train_subset = y_train

if X_test.shape[0] > 100:
    indices_test = np.random.choice(X_test.shape[0], 100, replace=False)
    X_test_subset = X_test[indices_test]
    y_test_subset = y_test[indices_test]
else:
    indices_test = np.arange(X_test.shape[0])
    X_test_subset = X_test
    y_test_subset = y_test

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="oob")

In [ ]:
rf_plus_mdi.explain(X=X_train, y=y_train)

In [ ]:
temp = rf_plus_mdi.explain(X=X_train, y=y_train)[0][:5]

In [ ]:
temp = np.abs(temp)
temp

In [ ]:
np.argsort(-1*temp)

In [ ]:
np.mean(np.argsort(-1*temp), axis=0)

In [ ]:
rf_plus_mdi.explain(X=X_train, y=y_train)[1][0]

In [ ]:
rf_plus_base.estimators_

In [ ]:
y_train[0]

In [ ]:
treeshap_fi, _, _, _ = tree_shap_evaluation_RF(X_train, y_train, X_train_subset, y_train_subset, X_test, y_test, X_test_subset, y_test_subset, fit=est, mode="absolute", train_only=True)

In [ ]:
lmdi_fi, _, _, _ = LFI_evaluation_RFPlus_oob_l2_norm_sign(X_train, y_train, X_train_subset, y_train_subset, X_test, y_test, X_test_subset, y_test_subset,  fit=rf_plus_base, mode="absolute", train_only=True)

In [ ]:
def select_top_features(array, sorted_indices, percentage):
    array = copy.deepcopy(array)
    num_features = array.shape[1]
    num_selected = int(np.ceil(num_features * percentage))
    selected_indices = sorted_indices[:num_selected]
    selected_array = array[:, selected_indices]
    return num_selected, selected_array

In [ ]:
mask_ratio = [0.05, 0.1, 0.25, 0.5, 0.9]
metric_results_shap_mse = []
metric_results_shap_r2 = []
train_fi_mean = np.mean(treeshap_fi, axis=0)
sorted_feature = np.argsort(-train_fi_mean)
for mask in mask_ratio:
    print(X_train.shape)
    num_features_masked, X_train_masked = select_top_features(X_train, sorted_feature, mask)
    print(X_train_masked.shape)
    num_features_masked, X_test_masked = select_top_features(X_test, sorted_feature, mask)
    print(X_test_masked.shape)
    ablation_models = {"RF_Regressor": RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42)}
                    #"Linear": LinearRegression(),
                    #"XGB_Regressor": xgb.XGBRegressor(random_state=42),
                    # 'Kernel_Ridge': KernelRidge(),
                    #"RF_Plus_Regressor": RandomForestPlusRegressor(rf_model=RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42))}
    # for a_model in ablation_models:
    #     ablation_models[a_model].fit(X_train_masked, y_train)
    rf = LinearRegression()# RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42)
    rf.fit(X_train_masked, y_train)
    y_pred = rf.predict(X_test_masked)
    metric_results_shap_mse.append(mean_squared_error(y_test, y_pred))
    metric_results_shap_r2.append(r2_score(y_test, y_pred))

In [ ]:
select_top_features(X_train, sorted_feature, 0.01)[1][0]

In [ ]:
select_top_features(X_test, sorted_feature, 0.01)[1][0]

In [ ]:
indices = np.nonzero(np.isin(X_train[0], select_top_features(X_train, sorted_feature, 0.01)[1][0]))[0]

print(indices)

In [ ]:
indices = np.nonzero(np.isin(X_test[0], select_top_features(X_test, sorted_feature, 0.01)[1][0]))[0]

print(indices)

In [ ]:
X_train[0]

In [ ]:
metric_results_shap_mse

In [ ]:
# plot metric_results_shap_r2
metric_results_shap_r2 = np.array(metric_results_shap_r2).reshape(len(mask_ratio), -1)
plt.figure()
plt.plot(mask_ratio, metric_results_shap_r2[:, 0], label="RF_Regressor")
plt.xlabel("Feature Ratio")
plt.ylabel("R2")
plt.legend()
plt.show()

In [ ]:
mask_ratio = [0.05, 0.1, 0.25, 0.5, 0.9]
metric_results_lmdi_mse = []
metric_results_lmdi_r2 = []
train_fi_mean = np.mean(local_fi_score_train, axis=0)
sorted_feature = np.argsort(-train_fi_mean)
for mask in mask_ratio:
    print(X_train.shape)
    num_features_masked, X_train_masked = select_top_features(X_train, sorted_feature, mask)
    print(X_train_masked.shape)
    num_features_masked, X_test_masked = select_top_features(X_test, sorted_feature, mask)
    print(X_test_masked.shape)
    ablation_models = {"RF_Regressor": RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42),
                    "Linear": LinearRegression(),
                    "XGB_Regressor": xgb.XGBRegressor(random_state=42),
                    # 'Kernel_Ridge': KernelRidge(),
                    "RF_Plus_Regressor": RandomForestPlusRegressor(rf_model=RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42))}
    for a_model in ablation_models:
        ablation_models[a_model].fit(X_train_masked, y_train)
        y_pred = ablation_models[a_model].predict(X_test_masked)
        metric_results_lmdi_mse.append(mean_squared_error(y_test, y_pred))
        metric_results_lmdi_r2.append(r2_score(y_test, y_pred))